# European Soccer Leagues Player in Starting XI Prediction

## Start Spark Session

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('appTest').getOrCreate()
spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
appid = spark._jsc.sc().applicationId()
print("You are working with", cores, "core(s) on appid: ", appid)
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1731918617409_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You are working with 4 core(s) on appid:  application_1731918617409_0001

## Import Libraries

In [ ]:
sc.install_pypi_package("pandas")
import pandas as pd

In [ ]:
sc.install_pypi_package("boto3")
import boto3

In [ ]:
sc.install_pypi_package("matplotlib")
import matplotlib.pyplot as plt

In [ ]:
sc.install_pypi_package("seaborn")
import seaborn as sns

In [ ]:
sc.install_pypi_package("pyarrow")
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
sc.install_pypi_package("fsspec")

In [ ]:
sc.install_pypi_package("sklearn")
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
sc.install_pypi_package("imblearn")
from imblearn.over_sampling import SMOTE

In [10]:
from pyspark.sql.functions import col, isnan, when, count, substring, when, concat, lit, year, month, dayofmonth, to_date, avg, coalesce
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DoubleType, BooleanType, DateType, FloatType
from pyspark.sql import Window

import io
from io import BytesIO

import numpy as np

import re

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load & Clean Data

In [70]:
# Path
path = "s3://final-project-gael-mota/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Appearances Data

In [71]:
# load Data
appearances = spark.read.csv(path+'datasets/appearances.csv', sep=',', header= True)

# Rename Variables
appearances = appearances.withColumnRenamed("date", "appearance_date")
         
# Print Schema
appearances.printSchema()
# appearances.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- appearance_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- player_club_id: string (nullable = true)
 |-- player_current_club_id: string (nullable = true)
 |-- appearance_date: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- minutes_played: string (nullable = true)

### Club Games Data

In [72]:
# Load Data
club_games = spark.read.csv(path+'datasets/club_games.csv', sep=',', header= True)

# Rename Variables
club_games = club_games.withColumnRenamed("own_position", "home_team_standings").withColumnRenamed("opponent_position", "away_team_standings").withColumnRenamed("own_goals", "goals_scored").withColumnRenamed("opponent_goals", "goals_conceded")
        
# Print Schema
club_games.printSchema()
# club_games.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- goals_scored: string (nullable = true)
 |-- home_team_standings: string (nullable = true)
 |-- own_manager_name: string (nullable = true)
 |-- opponent_id: string (nullable = true)
 |-- goals_conceded: string (nullable = true)
 |-- away_team_standings: string (nullable = true)
 |-- opponent_manager_name: string (nullable = true)
 |-- hosting: string (nullable = true)
 |-- is_win: string (nullable = true)

### Clubs Data

In [73]:
# Load Data
clubs = spark.read.csv(path+'datasets/clubs.csv', sep=',', header= True)

# Rename Variables
clubs = clubs.withColumnRenamed("name","club_name")

# Print Schema
clubs.printSchema()
# clubs.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- club_id: string (nullable = true)
 |-- club_code: string (nullable = true)
 |-- club_name: string (nullable = true)
 |-- domestic_competition_id: string (nullable = true)
 |-- total_market_value: string (nullable = true)
 |-- squad_size: string (nullable = true)
 |-- average_age: string (nullable = true)
 |-- foreigners_number: string (nullable = true)
 |-- foreigners_percentage: string (nullable = true)
 |-- national_team_players: string (nullable = true)
 |-- stadium_name: string (nullable = true)
 |-- stadium_seats: string (nullable = true)
 |-- net_transfer_record: string (nullable = true)
 |-- coach_name: string (nullable = true)
 |-- last_season: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- url: string (nullable = true)

### Competitions Data

In [74]:
# Load Data
competitions = spark.read.csv(path+'datasets/competitions.csv', sep=',', header= True)

# Rename Variables
competitions = competitions.withColumnRenamed("name","competition_name")
        
# Print Schema
competitions.printSchema()
# competitions.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- competition_id: string (nullable = true)
 |-- competition_code: string (nullable = true)
 |-- competition_name: string (nullable = true)
 |-- sub_type: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country_id: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- domestic_league_code: string (nullable = true)
 |-- confederation: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_major_national_league: string (nullable = true)

### Game Events Data

In [75]:
# Load Data
game_events = spark.read.csv(path+'datasets/game_events.csv', sep=',', header= True)

# Rename Variables
game_events = game_events.withColumnRenamed("type","event_type").withColumnRenamed("club_id","event_club_id")

# Print Schema
game_events.printSchema()
# game_events.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_event_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_club_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- player_in_id: string (nullable = true)
 |-- player_assist_id: string (nullable = true)

### Game Lineups Data

In [76]:
# Load Data
game_lineups = spark.read.csv(path+'datasets/game_lineups.csv', sep=',', header= True)

# Rename Variables
game_lineups = game_lineups.withColumnRenamed("type", "starting_status")
        
# Print Schema
game_lineups.printSchema()
# game_lineups.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_lineups_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- number: string (nullable = true)
 |-- team_captain: string (nullable = true)

### Games Data

In [77]:
# Load Data
games = spark.read.csv(path+'datasets/games.csv', sep=',', header= True)

# Rename Variables
games = games.withColumnRenamed("date","game_date").withColumnRenamed("home_club_goals", "scored_goals").withColumnRenamed("away_club_goals", "conceded_goals")

# Print schema
games.printSchema()
# games.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- round: string (nullable = true)
 |-- game_date: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_club_id: string (nullable = true)
 |-- scored_goals: string (nullable = true)
 |-- conceded_goals: string (nullable = true)
 |-- home_club_position: string (nullable = true)
 |-- away_club_position: string (nullable = true)
 |-- home_club_manager_name: string (nullable = true)
 |-- away_club_manager_name: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)
 |-- home_club_formation: string (nullable = true)
 |-- away_club_formation: string (nullable = true)
 |-- home_club_name: string (nullable = true)
 |-- away_club_name: string (nullable = true)
 |-- aggregate: string (nullable = true)
 |-- competition_t

### Player Valuations Data

In [78]:
# Load Data
player_valuations = spark.read.csv(path+'datasets/player_valuations.csv', sep=',', header= True)

# Rename Variables
player_valuations = player_valuations.withColumnRenamed("date","valuation_date")
        
# Print Schema
player_valuations.printSchema()
# player_valuations.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- player_id: string (nullable = true)
 |-- valuation_date: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- current_club_id: string (nullable = true)
 |-- player_club_domestic_competition_id: string (nullable = true)

### Players Data

In [79]:
# Load Data
players = spark.read.csv(path+'datasets/players.csv', sep=',', header= True)

# Rename Variables
players = players.withColumnRenamed("last_season","last_active_season").withColumnRenamed("name","full_name").withColumnRenamed("position","field_role")

# Print Schema
players.printSchema()
# players.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- player_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- last_active_season: string (nullable = true)
 |-- current_club_id: string (nullable = true)
 |-- player_code: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- city_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- field_role: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: string (nullable = true)
 |-- contract_expiration_date: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- url: string (nullable = true)
 |-- current_club_domestic_competition_id: string (nullable = true)
 |-- current_club_name: string (nullable = true)
 |-- market_value_in_eur: string (nullabl

### Transfers data

In [80]:
# Load Data
transfers = spark.read.csv(path+'datasets/transfers.csv', sep=',', header= True)

# Rename Variables
transfers = transfers.withColumnRenamed("market_value_in_eur","market_value")

# Print Schema
transfers.printSchema()
# transfers.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- player_id: string (nullable = true)
 |-- transfer_date: string (nullable = true)
 |-- transfer_season: string (nullable = true)
 |-- from_club_id: string (nullable = true)
 |-- to_club_id: string (nullable = true)
 |-- from_club_name: string (nullable = true)
 |-- to_club_name: string (nullable = true)
 |-- transfer_fee: string (nullable = true)
 |-- market_value: string (nullable = true)
 |-- player_name: string (nullable = true)

## Merge DataFrames

### Add Game Lineups Data

In [81]:
base_df = game_lineups.select('game_id', 'player_id', 'club_id', 'player_name', 'starting_status', 'position', 'team_captain')
base_df.printSchema()

total_rows = base_df.count()
rows_with_missing_values = base_df.filter(sum([col(c).isNull().cast("int") for c in base_df.columns]) > 0).count()
print(f"Out of {total_rows}, the are {rows_with_missing_values} rows with missing values.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: string (nullable = true)

Out of 2285289, the are 140070 rows with missing values.

### Add Appearances Data

In [82]:
df_w_appearances = base_df.join(appearances.select('game_id', 'player_id', 'minutes_played', 'goals', 'assists', 'yellow_cards', 'red_cards'), 
                                on=['game_id', 'player_id'], how='left')
df_w_appearances.printSchema()

total_rows = df_w_appearances.count()
rows_with_missing_values = df_w_appearances.filter(sum([col(c).isNull().cast("int") for c in df_w_appearances.columns]) > 0).count()
print(f"Out of {total_rows}, the are {rows_with_missing_values} rows with missing values.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)

Out of 2285289, the are 976127 rows with missing values.

### Add Players Data

In [83]:
df_w_players = df_w_appearances.join(players.select('player_id', 'field_role', 'height_in_cm', 'date_of_birth', 'foot', 'market_value_in_eur', 'last_active_season'),
                                     on='player_id', how='left')
df_w_players.printSchema()

total_rows = df_w_players.count()
rows_with_missing_values = df_w_players.filter(sum([col(c).isNull().cast("int") for c in df_w_players.columns]) > 0).count()
print(f"Out of {total_rows}, the are {rows_with_missing_values} rows with missing values.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- player_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)
 |-- field_role: string (nullable = true)
 |-- height_in_cm: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- last_active_season: string (nullable = true)

Out of 2285289, the are 995669 rows with missing values.

### Add Games Data

In [84]:
df_w_games = df_w_players.join(games.select('game_id', 'game_date', 'competition_id', 'season', 'competition_type', 'home_club_id',
                                            'away_club_id', 'home_club_formation', 'away_club_formation'), on='game_id', how='left')
df_w_games.printSchema()

total_rows = df_w_games.count()
rows_with_missing_values = df_w_games.filter(sum([col(c).isNull().cast("int") for c in df_w_games.columns]) > 0).count()
print(f"Out of {total_rows}, the are {rows_with_missing_values} rows with missing values.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)
 |-- field_role: string (nullable = true)
 |-- height_in_cm: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- last_active_season: string (nullable = true)
 |-- game_date: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- competition_type: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_

### Add Clubs Data

In [85]:
df_w_clubs = df_w_games.join(clubs.select('club_id', 'squad_size', 'average_age'), on='club_id', how='left')
df_w_clubs.printSchema()

total_rows = df_w_clubs.count()
rows_with_missing_values = df_w_clubs.filter(sum([col(c).isNull().cast("int") for c in df_w_clubs.columns]) > 0).count()
print(f"Out of {total_rows}, the are {rows_with_missing_values} rows with missing values.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- club_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)
 |-- field_role: string (nullable = true)
 |-- height_in_cm: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- last_active_season: string (nullable = true)
 |-- game_date: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- competition_type: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_

### Add Club Games Data

In [86]:
final_df = df_w_clubs.join(club_games.select('game_id', 'club_id', 'hosting', 'home_team_standings', 'away_team_standings', 'is_win'), 
                           on=['game_id', 'club_id'], how='left')
final_df.printSchema()

total_rows = final_df.count()
rows_with_missing_values = final_df.filter(sum([col(c).isNull().cast("int") for c in final_df.columns]) > 0).count()
print(f"Out of {total_rows}, the are {rows_with_missing_values} rows with missing values.")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)
 |-- field_role: string (nullable = true)
 |-- height_in_cm: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- last_active_season: string (nullable = true)
 |-- game_date: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- competition_type: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_

## Handle Missing Values

### Find the Initial Missing Values

In [87]:
# Identify the number of missing values per column
print("Final dataframe shape:", (final_df.count(), len(final_df.columns)))

null_counts = [(column, final_df.filter(col(column).isNull()).count()) 
               for column in final_df.columns]
initial_missing_values = spark.createDataFrame(null_counts, ["Feature", "Missing_Values"])
initial_missing_values.show(n=initial_missing_values.count(), truncate=False)

# Create a visualization of the initial missing values
initial_missing_values_p = initial_missing_values.toPandas()
initial_missing_values_p = initial_missing_values_p.sort_values('Missing_Values', ascending=True)

plt.figure(figsize=(12, 8))
bar_plot = sns.barplot(data=initial_missing_values_p, y='Feature', x='Missing_Values', orient ='h')
plt.title('Missing Values by Feature')
plt.xlabel('Number of Missing Values')
plt.ylabel('Features')
for index, value in enumerate(initial_missing_values_p['Missing_Values']):
    plt.text(value, index, str(value), color='black', va='center')
plt.tight_layout()

buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client = boto3.client('s3')
s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/InitialMissingValues.png",
    ContentType='image/png'
)

plt.close()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-87:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1493


Final dataframe shape: (2285289, 32)
+-------------------+--------------+
|Feature            |Missing_Values|
+-------------------+--------------+
|game_id            |93378         |
|club_id            |93378         |
|player_id          |93378         |
|player_name        |93378         |
|starting_status    |93378         |
|position           |93381         |
|team_captain       |140067        |
|minutes_played     |942966        |
|goals              |942966        |
|assists            |942966        |
|yellow_cards       |942966        |
|red_cards          |942966        |
|field_role         |342867        |
|height_in_cm       |365535        |
|date_of_birth      |343782        |
|foot               |376088        |
|market_value_in_eur|346325        |
|last_active_season |342867        |
|game_date          |97177         |
|competition_id     |97177         |
|season             |97177         |
|competition_type   |97177         |
|home_club_id       |97177         |
|

### Cast Data Types

In [88]:
# Interger Variables
interger_cols = ['minutes_played', 'goals', 'assists', 'yellow_cards', 'red_cards', 'height_in_cm',
                 'last_active_season', 'squad_size', 'home_team_standings', 'away_team_standings']
for column in interger_cols:
    final_df = final_df.withColumn(column, col(column).cast('int'))

# Float Variables
float_cols = ['market_value_in_eur', 'average_age']
for column in float_cols:
    final_df = final_df.withColumn(column, col(column).cast('float'))

# Date Variables
date_cols = ['game_date', 'date_of_birth']
for column in date_cols:
    final_df = final_df.withColumn(column, col(column).cast('date'))
    final_df = final_df.withColumn(column + '_day', dayofmonth(col(column)))
    final_df = final_df.withColumn(column + '_month', month(col(column)))
    final_df = final_df.withColumn(column + '_year', year(col(column)))
final_df = final_df.drop(*date_cols)

# Boolean Varaible
final_df = final_df.withColumn('is_win', col('is_win').cast(BooleanType()))
final_df = final_df.withColumn('team_captain', col('team_captain').cast(BooleanType()))

# Print Schema
final_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: string (nullable = true)
 |-- club_id: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- starting_status: string (nullable = true)
 |-- position: string (nullable = true)
 |-- team_captain: boolean (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- yellow_cards: integer (nullable = true)
 |-- red_cards: integer (nullable = true)
 |-- field_role: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- foot: string (nullable = true)
 |-- market_value_in_eur: float (nullable = true)
 |-- last_active_season: integer (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- competition_type: string (nullable = true)
 |-- home_club_id: string (nullable = true)
 |-- away_club_id: string (nullable = true)
 |-- home_club_formation: string (nullable = 

In [90]:
# Drop missing values from critical features
cleanedDF = final_df.dropna(subset=['player_id', 'game_id', 'club_id', 'competition_id', 'position', 
                                     'home_club_formation', 'away_club_formation', 'last_active_season'])

# Average depending on the position
window_spec = Window.partitionBy('position')
performance_cols = ['minutes_played', 'goals', 'assists', 'yellow_cards', 'red_cards', 'height_in_cm', 'market_value_in_eur']
for column in performance_cols:
    cleanedDF = cleanedDF.withColumn(column, coalesce(col(column), avg(col(column)).over(window_spec)))

# Median date of birth based on position
cleanedDF = cleanedDF.withColumn('date_of_birth_day', coalesce(col('date_of_birth_day'), avg(col('date_of_birth_day')).over(window_spec)))
cleanedDF = cleanedDF.withColumn('date_of_birth_month', coalesce(col('date_of_birth_month'), avg(col('date_of_birth_month')).over(window_spec)))
cleanedDF = cleanedDF.withColumn('date_of_birth_year', coalesce(col('date_of_birth_year'), avg(col('date_of_birth_year')).over(window_spec)))

# Global average
global_avg_cols = ['squad_size', 'average_age']
for col_name in global_avg_cols:
    avg_value = cleanedDF.select(avg(col(col_name))).first()[0]
    cleanedDF = cleanedDF.withColumn(col_name, coalesce(col(col_name), lit(avg_value)))

# Assume not team captain
cleanedDF = cleanedDF.withColumn('team_captain', 
    when(col('team_captain').isNull(), lit(False))
    .when(col('team_captain') == '1', lit(True))
    .when(col('team_captain') == '0', lit(False))
    .otherwise(lit(False)))
# cleanedDF = cleanedDF.withColumn('team_captain', when(col('team_captain').isNull(), lit(0)).otherwise(col('team_captain')))

# Assign Unknown to foot
cleanedDF = cleanedDF.withColumn('foot', coalesce(col('foot'), lit('Unknown')))

# Average standings depending on competiton and season
global_window = Window.partitionBy()
standings_window = Window.partitionBy('competition_id', 'season')
cleanedDF = cleanedDF.withColumn('home_team_standings', when(col('home_team_standings').isNull(), coalesce(
            avg(col('home_team_standings')).over(standings_window), avg(col('home_team_standings')).over(global_window))).otherwise(col('home_team_standings')))
cleanedDF = cleanedDF.withColumn('away_team_standings', when(col('away_team_standings').isNull(), coalesce(
            avg(col('away_team_standings')).over(standings_window), avg(col('away_team_standings')).over(global_window))).otherwise(col('home_team_standings')))

# See the missing values per row after this changes
print("Final dataframe shape:", (cleanedDF.count(), len(cleanedDF.columns)))
null_counts = [(column, cleanedDF.filter(col(column).isNull()).count()) 
               for column in cleanedDF.columns]
remaining_missing_values = spark.createDataFrame(null_counts, ["Feature", "Missing_Values"])
remaining_missing_values.show(n=remaining_missing_values.count(), truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-90:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1993


Final dataframe shape: (1937749, 36)
+-------------------+--------------+
|Feature            |Missing_Values|
+-------------------+--------------+
|game_id            |0             |
|club_id            |0             |
|player_id          |0             |
|player_name        |0             |
|starting_status    |0             |
|position           |0             |
|team_captain       |0             |
|minutes_played     |0             |
|goals              |0             |
|assists            |0             |
|yellow_cards       |0             |
|red_cards          |0             |
|field_role         |0             |
|height_in_cm       |0             |
|foot               |0             |
|market_value_in_eur|0             |
|last_active_season |0             |
|competition_id     |0             |
|season             |0             |
|competition_type   |0             |
|home_club_id       |0             |
|away_club_id       |0             |
|home_club_formation|0             |
|

## Athena

### Create the Table
The command below was not creating the table for some reason. I printed the query, and copy/paste it to the athena editor and it worked. 

In [64]:
# Athena configuration
DATABASE_NAME = 'default'
TABLE_NAME = 'FinalProjectGael'
S3_BUCKET_NAME = 'final-project-gael-mota'
AWS_REGION = 'us-east-1'

# Initialize Athena client (no need to specify credentials)
athena_client = boto3.client('athena', region_name=AWS_REGION)

# Define the table creation query
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `{DATABASE_NAME}`.`{TABLE_NAME}` (
    game_id STRING,
    club_id STRING,
    player_id STRING,
    player_name STRING,
    starting_status STRING,
    position STRING,
    team_captain BOOLEAN,
    minutes_played INT,
    goals INT,
    assists INT,
    yellow_cards INT,
    red_cards INT,
    field_role STRING,
    height_in_cm INT,
    foot STRING,
    market_value_in_eur DOUBLE,
    last_active_season INT,
    competition_id STRING,
    season STRING,
    competition_type STRING,
    home_club_id STRING,
    away_club_id STRING,
    home_club_formation STRING,
    away_club_formation STRING,
    squad_size INT,
    average_age DOUBLE,
    hosting STRING,
    home_team_standings INT,
    away_team_standings INT,
    is_win BOOLEAN,
    game_date_day INT,
    game_date_month INT,
    game_date_year INT,
    date_of_birth_day INT,
    date_of_birth_month INT,
    date_of_birth_year INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION 's3://{S3_BUCKET_NAME}/athena'
"""

print(create_table_query.strip())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CREATE EXTERNAL TABLE IF NOT EXISTS `default`.`FinalProjectGael` (
    game_id STRING,
    club_id STRING,
    player_id STRING,
    player_name STRING,
    starting_status STRING,
    position STRING,
    team_captain BOOLEAN,
    minutes_played INT,
    goals INT,
    assists INT,
    yellow_cards INT,
    red_cards INT,
    field_role STRING,
    height_in_cm INT,
    foot STRING,
    market_value_in_eur DOUBLE,
    last_active_season INT,
    competition_id STRING,
    season STRING,
    competition_type STRING,
    home_club_id STRING,
    away_club_id STRING,
    home_club_formation STRING,
    away_club_formation STRING,
    squad_size INT,
    average_age DOUBLE,
    hosting STRING,
    home_team_standings INT,
    away_team_standings INT,
    is_win BOOLEAN,
    game_date_day INT,
    game_date_month INT,
    game_date_year INT,
    date_of_birth_day INT,
    date_of_birth_month INT,
    date_of_birth_year INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION 's3://fina

### Save to CSV file

In [92]:
# Write the dataframe to a CSV file
output_path = 's3://final-project-gael-mota/athena/cleaned_soccer_data'
cleanedDF.write.csv(output_path, header=False, mode="overwrite")
print("Output saved to S3!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output saved to S3!

## Data Exploration

### Starting Status Distribution

In [21]:
starting_status = spark.read.csv("s3://final-project-gael-mota/athena/exploratory_tables/starting_status_distribution_raw/*.gz", 
                        header=False, inferSchema=True)

# Converting to Pandas DF
starting_status_pandas = starting_status.toPandas()

# Plotting a histogram
plt.figure(figsize=(12, 8))
bar_plot = sns.barplot(data=starting_status_pandas, y='_c1', x='_c0')
plt.title("Starting XI Player's Status Distribution")
plt.xlabel('Status')
plt.ylabel('Count')
plt.tight_layout()

buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client = boto3.client('s3')
s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/StartingStatus.png",
    ContentType='image/png'
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'QESFEB8F7M7JXZN4', 'HostId': 'P7S3WKHqQZpQEjXkc4KEwncpl6uH0DafMKGvjMVQz4KTtWR55kwMQR/LCv1Sa2QGgzpsM72pk5qB6PFOEW2RUw==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'P7S3WKHqQZpQEjXkc4KEwncpl6uH0DafMKGvjMVQz4KTtWR55kwMQR/LCv1Sa2QGgzpsM72pk5qB6PFOEW2RUw==', 'x-amz-request-id': 'QESFEB8F7M7JXZN4', 'date': 'Mon, 18 Nov 2024 05:30:01 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"0d1f8e66a6c595e6532d53ce9e180cac"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"0d1f8e66a6c595e6532d53ce9e180cac"', 'ServerSideEncryption': 'AES256'}

### Player's Age Distribution

In [22]:
# Loading the data
age_df = spark.read.csv("s3://final-project-gael-mota/athena/exploratory_tables/players_age_distribution/*.gz", 
                        header=False, inferSchema=True)

# Converting to Pandas DF
age_pandas = age_df.toPandas()

# Plotting a histogram
plt.figure(figsize=(12, 8))
bar_plot = sns.barplot(data=age_pandas, y='_c1', x='_c0')
plt.title("Player's Age Distribution")
plt.xlabel('Age')
plt.ylabel('Count')
plt.tight_layout()

buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client = boto3.client('s3')
s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/AgeDistribution.png",
    ContentType='image/png'
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'T7CAE4VNG6B5VQJQ', 'HostId': '/8fLWthEw6rKrKfuSWxuxnkrqyf1i7eeQRjawclCnUPd2fKzcPbC0J1v3+11BYRYOdXnXZZhSMKVgFUCZvrJaFyVS5toZQ9e2xWCxn46S4I=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '/8fLWthEw6rKrKfuSWxuxnkrqyf1i7eeQRjawclCnUPd2fKzcPbC0J1v3+11BYRYOdXnXZZhSMKVgFUCZvrJaFyVS5toZQ9e2xWCxn46S4I=', 'x-amz-request-id': 'T7CAE4VNG6B5VQJQ', 'date': 'Mon, 18 Nov 2024 05:30:04 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"ba4096daf405eb3799090475619abf37"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"ba4096daf405eb3799090475619abf37"', 'ServerSideEncryption': 'AES256'}

### Active Player's Age Distribution

In [23]:
# Loading the data
active_age_df = spark.read.csv("s3://final-project-gael-mota/athena/exploratory_tables/last_active_season_players_age_distribution/*.gz", 
                        header=False, inferSchema=True)

# Converting to Pandas DF
active_age_pandas = active_age_df.toPandas()

# Plotting a histogram
plt.figure(figsize=(12, 8))
bar_plot = sns.barplot(data=active_age_pandas, y='_c1', x='_c0')
plt.title("Active Player's Age Distribution")
plt.xlabel('Age')
plt.ylabel('Count')
plt.tight_layout()

buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client = boto3.client('s3')
s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/ActivePlayersAgeDistribution.png",
    ContentType='image/png'
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'MCZG0FJSRNXKV0S8', 'HostId': 'BAWPe9pVL4UJcqGI+jMhmR2lQAxMJL6B0GhuZkK0DmWGXOc8pcqOyCb+BqnZ4KzBYoScbCsWNIe9jSTfj3hVr2IT4Tydofuf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'BAWPe9pVL4UJcqGI+jMhmR2lQAxMJL6B0GhuZkK0DmWGXOc8pcqOyCb+BqnZ4KzBYoScbCsWNIe9jSTfj3hVr2IT4Tydofuf', 'x-amz-request-id': 'MCZG0FJSRNXKV0S8', 'date': 'Mon, 18 Nov 2024 05:30:10 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"f8d9197ce8257206cc8a4f65737ebe21"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"f8d9197ce8257206cc8a4f65737ebe21"', 'ServerSideEncryption': 'AES256'}

## ML Classifier Training

In [11]:
# Difine schema
schema = StructType([
    StructField("game_id", StringType(), True),
    StructField("club_id", StringType(), True),
    StructField("player_id", StringType(), True),
    StructField("player_name", StringType(), True),
    StructField("starting_status", StringType(), True),
    StructField("position", StringType(), True),
    StructField("team_captain", BooleanType(), True),
    StructField("minutes_played", IntegerType(), True),
    StructField("goals", IntegerType(), True),
    StructField("assists", IntegerType(), True),
    StructField("yellow_cards", IntegerType(), True),
    StructField("red_cards", IntegerType(), True),
    StructField("field_role", StringType(), True),
    StructField("height_in_cm", IntegerType(), True),
    StructField("foot", StringType(), True),
    StructField("market_value_in_eur", FloatType(), True),
    StructField("last_active_season", IntegerType(), True),
    StructField("competition_id", StringType(), True),
    StructField("season", StringType(), True),
    StructField("competition_type", StringType(), True),
    StructField("home_club_id", StringType(), True),
    StructField("away_club_id", StringType(), True),
    StructField("home_club_formation", StringType(), True),
    StructField("away_club_formation", StringType(), True),
    StructField("squad_size", IntegerType(), True),
    StructField("average_age", FloatType(), True),
    StructField("hosting", StringType(), True),
    StructField("home_team_standings", IntegerType(), True),
    StructField("away_team_standings", IntegerType(), True),
    StructField("is_win", BooleanType(), True),
    StructField("game_date_day", IntegerType(), True),
    StructField("game_date_month", IntegerType(), True),
    StructField("game_date_year", IntegerType(), True),
    StructField("date_of_birth_day", IntegerType(), True),
    StructField("date_of_birth_month", IntegerType(), True),
    StructField("date_of_birth_year", IntegerType(), True)
])

# Loading the data
df = spark.read.csv("s3://final-project-gael-mota/athena/europe_players_2024/*.gz", header=False, schema=schema)

# Converting to Pandas DF
df = df.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature Engineering

In [12]:
# Create game_date
df['game_date'] = pd.to_datetime(
    df[['game_date_year', 'game_date_month', 'game_date_day']].astype(str).agg('-'.join, axis=1))

# Sort data by player and date (important for rolling statistics)
df = df.sort_values(['player_id', 'game_date'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# Performance metrics
df['goals'] = df['goals'].fillna(0)

# Mask to calculate the metrics per game
minutes_played_mask = df['minutes_played'].astype(float) > 0

# Total minutes played
df['total_minutes_played'] = df.groupby('player_id')['minutes_played'].transform('sum')

# Goals per game
df['goals_per_game'] = np.where(minutes_played_mask, df['goals'].astype(float) * 90 / df['minutes_played'].astype(float), 0)
# Total Goals 
df['total_goals'] = df.groupby('player_id')['goals'].transform('sum')

# Assists per game
df['assists_per_game'] = np.where(minutes_played_mask, df['assists'].astype(float) * 90 / df['minutes_played'].astype(float), 0)
# Total assists
df['total_assists'] = df.groupby('player_id')['assists'].transform('sum')

# Contributions (goals + assits)
df['goal_contributions'] = df['goals'] + df['assists']

# Cards per game
df['cards_per_game'] = np.where(minutes_played_mask, (df['yellow_cards'].astype(float) + df['red_cards'].astype(float) * 2) * 90 / df['minutes_played'].astype(float), 0)
# Total cards
df['total_cards'] = df.groupby('player_id')['yellow_cards'].transform('sum') + df.groupby('player_id')['red_cards'].transform('sum')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Starting XI percentage in previous games
df['is_starting'] = (df['starting_status'] == 'starting_lineup').astype(int)
window_size = 5

# Percentage of previous games as starter
df['prev_starts_pct'] = df.groupby('player_id')['is_starting'].transform(
    lambda x: x.rolling(window=window_size, min_periods=1).mean()).shift(1).fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Age calculation 
df['age'] = (df['game_date_year'].astype(int) - df['date_of_birth_year'].astype(int))

# Age categorization
df['age_category'] = pd.cut(df['age'], bins=[0, 22, 30, float('inf')], labels=['rookie', 'prime', 'older'], right=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Market value tiers
df['market_value_in_eur'] = pd.to_numeric(df['market_value_in_eur'])
df['market_value_tier'] = pd.qcut(df['market_value_in_eur'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Formation compatibility to position
df['home_club_formation'] = df['home_club_formation'].apply(lambda x: re.search(r'\d-\d-\d(?:-\d)?', x).group(0))

defenders = np.array([int(f.split('-')[0]) for f in df['home_club_formation']])
forwards = np.array([int(f.split('-')[-1]) for f in df['home_club_formation']])
midfielders = np.array([
    int(f.split('-')[1]) if len(f.split('-')) == 3 
    else int(f.split('-')[1]) + int(f.split('-')[2]) 
    for f in df['home_club_formation']
])

# Calculate compatibility
df['formation_compatibility'] = np.select(
    condlist=[
        df['field_role'] == 'goalkeeper',
        df['field_role'] == 'Defender',
        df['field_role'] == 'Midfield',
        df['field_role'] == 'Attack'],
    choicelist=[
        1.0,
        defenders / 11,
        midfielders / 11,
        forwards / 11], default=0.5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Competition type importance
competition_importance = {
    'international_cup': 1,  # Most Important
    'domestic_league': 2,
    'domestic_cup': 3,
    'other': 4,             # Least Important
}
df['competition_importance'] = df['competition_type'].map(competition_importance)

df['competition_importance'] = pd.Categorical(
    df['competition_importance'],
    categories=[1, 2, 3, 4],
    ordered=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Training Model

In [24]:
df.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Index: 556749 entries, 315090 to 125690
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   game_id                  556749 non-null  object        
 1   club_id                  556749 non-null  object        
 2   player_id                556749 non-null  object        
 3   player_name              556749 non-null  object        
 4   starting_status          556749 non-null  object        
 5   position                 556749 non-null  object        
 6   team_captain             556749 non-null  bool          
 7   minutes_played           556749 non-null  int32         
 8   goals                    556749 non-null  float64       
 9   assists                  556749 non-null  int32         
 10  yellow_cards             556749 non-null  int32         
 11  red_cards                556749 non-null  int32         
 12  field_role      

In [25]:
# Select features for modeling
features = [
    'team_captain', 'minutes_played', 'field_role', 'height_in_cm', 'foot',
    'squad_size', 'average_age', 'hosting', 'is_win', 'total_minutes_played',
    'goals_per_game', 'total_goals', 'assists_per_game', 'total_assists', 
    'goal_contributions', 'cards_per_game', 'total_cards', 'prev_starts_pct', 
    'age', 'age_category', 'market_value_tier', 'formation_compatibility', 
    'competition_importance', 'is_starting'  # target variable
]

# Keep only the desired features
df_model = df[features].copy()
df_model.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Index: 556749 entries, 315090 to 125690
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   team_captain             556749 non-null  bool    
 1   minutes_played           556749 non-null  int32   
 2   field_role               556749 non-null  object  
 3   height_in_cm             556749 non-null  int32   
 4   foot                     556749 non-null  object  
 5   squad_size               556749 non-null  int32   
 6   average_age              556749 non-null  float32 
 7   hosting                  556749 non-null  object  
 8   is_win                   556749 non-null  bool    
 9   total_minutes_played     556749 non-null  int32   
 10  goals_per_game           556749 non-null  float64 
 11  total_goals              556749 non-null  float64 
 12  assists_per_game         556749 non-null  float64 
 13  total_assists            556749 non-null  in

In [28]:
# Cast Int variables
boolean_columns = ['total_goals', 'goal_contributions']
for column in boolean_columns:
    df_model[column] = df_model[column].astype(int)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Encode Categorical Variables 
categorical_columns = ['team_captain', 'field_role', 'foot', 'hosting', 'is_win', 'age_category',
                       'market_value_tier', 'competition_importance', 'is_starting']

encoder = LabelEncoder()
for column in categorical_columns:
    df_model[column] = encoder.fit_transform(df_model[column].astype(str))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_model.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Index: 556749 entries, 315090 to 125690
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   team_captain             556749 non-null  int64  
 1   minutes_played           556749 non-null  int32  
 2   field_role               556749 non-null  int64  
 3   height_in_cm             556749 non-null  int32  
 4   foot                     556749 non-null  int64  
 5   squad_size               556749 non-null  int32  
 6   average_age              556749 non-null  float32
 7   hosting                  556749 non-null  int64  
 8   is_win                   556749 non-null  int64  
 9   total_minutes_played     556749 non-null  int32  
 10  goals_per_game           556749 non-null  float64
 11  total_goals              556749 non-null  int64  
 12  assists_per_game         556749 non-null  float64
 13  total_assists            556749 non-null  int32  
 14  goal

In [38]:
# Split features and target
X = df_model.drop('is_starting', axis=1)
y = df_model['is_starting']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Initialize models with more trees and balanced class weights
rf_params = {
    'n_estimators': 100,
    'max_depth': 15,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'class_weight': 'balanced',
    'random_state': 42
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# Train and evaluate without SMOTE
rf_no_smote = RandomForestClassifier(**rf_params)
cv_scores_no_smote = cross_val_score(rf_no_smote, X_train, y_train, cv=5, scoring='balanced_accuracy')
rf_no_smote.fit(X_train, y_train)
y_pred_no_smote = rf_no_smote.predict(X_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
# Print results
print("Without SMOTE:")
print("Cross-validation balanced accuracy scores:", cv_scores_no_smote)
print("Average CV balanced accuracy score:", cv_scores_no_smote.mean())
print("\nClassification Report:")
print(classification_report(y_test, y_pred_no_smote))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Without SMOTE:
Cross-validation balanced accuracy scores: [0.93939609 0.93972641 0.93805405 0.93957929 0.9392188 ]
Average CV balanced accuracy score: 0.9391949298273113

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.98      0.91     42940
           1       0.99      0.90      0.94     68410

    accuracy                           0.93    111350
   macro avg       0.92      0.94      0.93    111350
weighted avg       0.94      0.93      0.93    111350

In [69]:
# Train and evaluate with SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
rf_smote = RandomForestClassifier(**rf_params)
cv_scores_smote = cross_val_score(rf_smote, X_train_smote, y_train_smote, cv=5, scoring='balanced_accuracy')
rf_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = rf_smote.predict(X_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
print("With SMOTE:")
print("Cross-validation balanced accuracy scores:", cv_scores_smote)
print("Average CV balanced accuracy score:", cv_scores_smote.mean())
print("\nClassification Report:")
print(classification_report(y_test, y_pred_smote))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

With SMOTE:
Cross-validation balanced accuracy scores: [0.93449498 0.93514331 0.93593773 0.94996163 0.9500804 ]
Average CV balanced accuracy score: 0.941123608250242

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91     42940
           1       0.97      0.91      0.94     68410

    accuracy                           0.93    111350
   macro avg       0.92      0.94      0.93    111350
weighted avg       0.93      0.93      0.93    111350

In [71]:
# Create a figure with two subplots side by side
plt.figure(figsize=(15, 6))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot original class distribution
original_dist = pd.Series(y_train).value_counts()
sns.barplot(x=original_dist.index, y=original_dist.values, ax=ax1)
ax1.set_title('Original Distribution (Before Balancing)')
ax1.set_xlabel('Starting Status (0=Substitute, 1=Starting)')
ax1.set_ylabel('Count')

# Plot SMOTE-balanced class distribution
smote_dist = pd.Series(y_train_smote).value_counts()
sns.barplot(x=smote_dist.index, y=smote_dist.values, ax=ax2)
ax2.set_title('Class Distribution After SMOTE')
ax2.set_xlabel('Starting Status (0=Substitute, 1=Starting)')
ax2.set_ylabel('Count')

plt.tight_layout()

# Save the plot
buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client = boto3.client('s3')
s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/ClassBalanceDistribution.png",
    ContentType='image/png'
)

# Print the actual numbers
print("Class distribution before SMOTE:")
print(original_dist)
print("\nClass distribution after SMOTE:")
print(smote_dist)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Class distribution before SMOTE:
is_starting
1    273641
0    171758
Name: count, dtype: int64

Class distribution after SMOTE:
is_starting
1    273641
0    273641
Name: count, dtype: int64

In [72]:
# Create figure for side-by-side comparison
plt.figure(figsize=(20, 8))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Prepare feature importance data for both models
importance_no_smote = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_no_smote.feature_importances_
}).sort_values('importance', ascending=True)  # Ascending for horizontal bars

importance_smote = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_smote.feature_importances_
}).sort_values('importance', ascending=True)  # Ascending for horizontal bars

# Plot feature importance without SMOTE
sns.barplot(data=importance_no_smote, x='importance', y='feature', ax=ax1)
ax1.set_title('Feature Importance (Without SMOTE)')
ax1.set_xlabel('Importance Score')
ax1.set_ylabel('Features')

# Plot feature importance with SMOTE
sns.barplot(data=importance_smote, x='importance', y='feature', ax=ax2)
ax2.set_title('Feature Importance (With SMOTE)')
ax2.set_xlabel('Importance Score')
ax2.set_ylabel('Features')

plt.tight_layout()

# Save the plot
buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client = boto3.client('s3')
s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/FeatureImportance_Comparison.png",
    ContentType='image/png'
)
plt.close()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
# Create DataFrames for both models
importance_no_smote_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance_No_SMOTE': rf_no_smote.feature_importances_
}).sort_values('Importance_No_SMOTE', ascending=False)

importance_smote_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance_SMOTE': rf_smote.feature_importances_
}).sort_values('Importance_SMOTE', ascending=False)

# Merge the two DataFrames
comparison_df = pd.merge(
    importance_no_smote_df, 
    importance_smote_df, 
    on='Feature', 
    suffixes=('_No_SMOTE', '_SMOTE')
)

# Calculate ranks (using method='min' to get integer ranks)
comparison_df['Rank_No_SMOTE'] = range(1, len(comparison_df) + 1)
comparison_df['Rank_SMOTE'] = comparison_df['Importance_SMOTE'].rank(ascending=False, method='min').astype(int)
comparison_df['Rank_Change'] = comparison_df['Rank_No_SMOTE'] - comparison_df['Rank_SMOTE']

# Format importance scores to 4 decimal places
comparison_df['Importance_No_SMOTE'] = comparison_df['Importance_No_SMOTE'].map('{:.4f}'.format)
comparison_df['Importance_SMOTE'] = comparison_df['Importance_SMOTE'].map('{:.4f}'.format)

# Reorder columns for better readability
comparison_df = comparison_df[[
    'Feature', 
    'Importance_No_SMOTE', 
    'Rank_No_SMOTE',
    'Importance_SMOTE', 
    'Rank_SMOTE',
    'Rank_Change'
]]

print("Feature Importance Comparison:")
print("\nPositive Rank Change means feature became more important after SMOTE")
print("Negative Rank Change means feature became less important after SMOTE")
print("\n", comparison_df.to_string(index=False))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Feature Importance Comparison:

Positive Rank Change means feature became more important after SMOTE
Negative Rank Change means feature became less important after SMOTE

                 Feature Importance_No_SMOTE  Rank_No_SMOTE Importance_SMOTE  Rank_SMOTE  Rank_Change
         minutes_played              0.6281              1           0.5860           1            0
        prev_starts_pct              0.1655              2           0.2187           2            0
         cards_per_game              0.0359              3           0.0334           3            0
     goal_contributions              0.0225              4           0.0296           4            0
         goals_per_game              0.0176              5           0.0152           5            0
            total_cards              0.0164              6           0.0134           6            0
   total_minutes_played              0.0136              7           0.0121           8           -1
           team_capt

In [74]:
# Plot confusion matrices
plt.figure(figsize=(15, 6))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
# Without SMOTE
cm_no_smote = confusion_matrix(y_test, y_pred_no_smote)
sns.heatmap(cm_no_smote, annot=True, fmt='d', cmap='Blues', ax=ax1)
ax1.set_title('Confusion Matrix - Without SMOTE')
ax1.set_ylabel('True Label (0=Substitute, 1=Starting)')
ax1.set_xlabel('Predicted Label (0=Substitute, 1=Starting)')

# With SMOTE
cm_smote = confusion_matrix(y_test, y_pred_smote)
sns.heatmap(cm_smote, annot=True, fmt='d', cmap='Blues', ax=ax2)
ax2.set_title('Confusion Matrix - With SMOTE')
ax2.set_ylabel('True Label (0=Substitute, 1=Starting)')
ax2.set_xlabel('Predicted Label (0=Substitute, 1=Starting)')

plt.tight_layout()

# Save the confusion matrices plot
buffer = io.BytesIO()
plt.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
buffer.seek(0)
img_data = buffer.getvalue()

s3_client.put_object(
    Body=img_data,
    Bucket="final-project-gael-mota",
    Key="visualizations/ConfusionMatrices_Updated.png",
    ContentType='image/png'
)
plt.close()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…